In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
import string
import re
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
# from textstat.textstat import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from keras.optimizers import Adam

/home/bhargav/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
embeddings_index = dict()
f = open('../glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [3]:
df = pd.read_csv("../Proper Experiments/Parsed Datasets/hatEval_2019.csv")
df.text=df.text.astype(str)
df.head(5)

,Unnamed: 0,text,class
0,0,hurray saving many ways potus realdonaldtrump ...,0
1,1,would young fighting age men vast majority one...,0
2,2,kamalaharris illegals dump kids border like r...,0
3,3,times : nearly white states pose an array prob...,0
4,4,orban brussels : european leaders ignoring peo...,0


In [4]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
vocabulary_size=len(tokenizer.word_index) + 1
print(vocabulary_size)

embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

24759


In [5]:
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    break

In [7]:
hidden_size=[75,100,125]
learning_rate=[0.0001, 0.001, 0.01]
score=[]
accuracy=[]

for i in learning_rate:
    sco=[]
    acc=[]
    for j in hidden_size:
        
        model_glove = Sequential()
        model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
        model_glove.add(LSTM(300))
        model_glove.add(Dense(j, activation='relu'))
        model_glove.add(Dense(1, activation='sigmoid'))
        model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=i), metrics=['accuracy'])
        
        model_glove.fit(Xtrain, np.array(ytrain), epochs = 3)

        pred=model_glove.predict_classes(X_val)
        
        acc.append(accuracy_score(y_val, pred))
        sco.append(f1_score(y_val, pred, average='macro'))
    score.append(sco)
    accuracy.append(acc)

Epoch 1/3
6472/6472 [==============================] - 38s 6ms/step - loss: 0.4402 - acc: 0.8288
Epoch 2/3
6472/6472 [==============================] - 35s 5ms/step - loss: 0.3820 - acc: 0.8361
Epoch 3/3
6472/6472 [==============================] - 37s 6ms/step - loss: 0.3650 - acc: 0.8413
Epoch 1/3
6472/6472 [==============================] - 40s 6ms/step - loss: 0.4446 - acc: 0.8257
Epoch 2/3
6472/6472 [==============================] - 36s 6ms/step - loss: 0.3850 - acc: 0.8368
Epoch 3/3
6472/6472 [==============================] - 34s 5ms/step - loss: 0.3636 - acc: 0.8439
Epoch 1/3
6472/6472 [==============================] - 35s 5ms/step - loss: 0.4479 - acc: 0.8211
Epoch 2/3
6472/6472 [==============================] - 36s 6ms/step - loss: 0.3834 - acc: 0.8388
Epoch 3/3
6472/6472 [==============================] - 33s 5ms/step - loss: 0.3660 - acc: 0.8421
Epoch 1/3
6472/6472 [==============================] - 33s 5ms/step - loss: 0.4113 - acc: 0.8277
Epoch 2/3
6472/6472 [=========

In [8]:
print(score)

[[0.5946659349377683, 0.5943526086903559, 0.6114285714285714], [0.6114285714285714, 0.6233427079148253, 0.6669072773804054], [0.6627855188649234, 0.6012031399016946, 0.48957464553794827]]


In [9]:
ac=[]
sc=[]
i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
    model_glove.add(LSTM(300))
    model_glove.add(Dense(75, activation='relu'))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
    
    print("Fold Number: "+str(i))  
    model_glove.fit(Xtrain, np.array(ytrain), epochs = 5)
    
    pred=model_glove.predict_classes(Xtest)
    
    a=accuracy_score(ytest, pred)
    s=f1_score(ytest, pred, average='macro')
    
    if i==0:
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_hatEval_2019.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_hatEval_2019.h5")
        print("Saved model to disk")
    elif s>max(sc):
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_hatEval_2019.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_hatEval_2019.h5")
        print("Saved model to disk")

    ac.append(a)
    sc.append(s)
    i=i+1    

Fold Number: 0
Epoch 1/5
6472/6472 [==============================] - 33s 5ms/step - loss: 0.4510 - acc: 0.8194
Epoch 2/5
6472/6472 [==============================] - 32s 5ms/step - loss: 0.3990 - acc: 0.8307
Epoch 3/5
6472/6472 [==============================] - 34s 5ms/step - loss: 0.3553 - acc: 0.8460
Epoch 4/5
6472/6472 [==============================] - 35s 5ms/step - loss: 0.3233 - acc: 0.8636
Epoch 5/5
6472/6472 [==============================] - 33s 5ms/step - loss: 0.2791 - acc: 0.8829
Saved model to disk
Fold Number: 1
Epoch 1/5
6473/6473 [==============================] - 31s 5ms/step - loss: 0.4203 - acc: 0.8220
Epoch 2/5
6473/6473 [==============================] - 30s 5ms/step - loss: 0.3673 - acc: 0.8383
Epoch 3/5
6473/6473 [==============================] - 30s 5ms/step - loss: 0.3367 - acc: 0.8554
Epoch 4/5
6473/6473 [==============================] - 30s 5ms/step - loss: 0.3174 - acc: 0.8634
Epoch 5/5
6473/6473 [==============================] - 30s 5ms/step - loss: 0

In [10]:
print("Accuracies: ")
print(ac)
print("Mean: "+str(np.mean(np.array(ac))))
print("Std Dev: "+str(np.std(np.array(ac))))

Accuracies: 
[0.840024706609018, 0.8127317676143387, 0.8207663782447466, 0.8201483312731768, 0.8121137206427689]
Mean: 0.8211569808768097
Std Dev: 0.010098770152088994


In [11]:
print("Macro-F1-Scores: ")
print(sc)
print("Mean: "+str(np.mean(np.array(sc))))
print("Std Dev: "+str(np.std(np.array(sc))))

Macro-F1-Scores: 
[0.7000404170466943, 0.6195201452835241, 0.5933348586959537, 0.5748575824640197, 0.6631355177694316]
Mean: 0.6301777042519247
Std Dev: 0.04582207070256412


In [5]:
# load json and create model
json_file = open('Models/model_hatEval_2019.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_glove = model_from_json(loaded_model_json)
# load weights into new model
model_glove.load_weights("Models/model_hatEval_2019.h5")
print("Loaded model from disk")

Loaded model from disk


In [6]:
# Dataset-1
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      1
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      1
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      1
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      1


In [7]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_hatEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    hatEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    hatEval_2019_model.load_weights("Models/model_hatEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in hatEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    hatEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    hatEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=hatEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f4c8bcd0a90> False
<keras.layers.recurrent.LSTM object at 0x7f4c8bcd0ba8> True
<keras.layers.core.Dense object at 0x7f4c7cf78b00> True
<keras.layers.core.Dense object at 0x7f4c7cee0710> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f4c8bcd0a90> False
<keras.layers.recurrent.LSTM object at 0x7f4c8bcd0ba8> False
<keras.layers.core.Dense object at 0x7f4c7cf78b00> True
<keras.layers.core.Dense object at 0x7f4c7cee0710> True
Epoch 1/8
4460/4460 [==============================] - 9s 2ms/step - loss: 0.8812 - acc: 0.5381
Epoch 2/8
4460/4460 [==============================] - 8s 2ms/step - loss: 0.3966 - acc: 0.8334
Epoch 3/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.3641 - acc: 0.8388
Epoch 4/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.3402 - acc: 0.8471
Epoch 5/8
4460/4460 [==============================] - 7s 2ms/step - loss: 

In [8]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.845360824742268, 0.8435679067682653, 0.8437219730941704, 0.854932735426009, 0.8573991031390135]
Mean: 0.8489965086339453
Std Dev: 0.005938872345488338
Macro-F1-Scores: 
[0.6439361662264992, 0.6214194721590136, 0.6265831763625764, 0.6806025740004372, 0.6894232032584418]
Mean: 0.6523929184013937
Std Dev: 0.027799426178244234


In [9]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.2025997310623039, 0.20730614074406095, 0.20426008968609866, 0.20717488789237667, 0.20538116591928252]
Mean: 0.20534440306082455
Std Dev: 0.0017837825449424597
Macro-F1-Scores: 
[0.19162073265888094, 0.19758985534281628, 0.1939256209394968, 0.1980401346064973, 0.19534895749230435]
Mean: 0.19530506020799915
Std Dev: 0.002374024849476144


In [10]:
# Dataset-2
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Waseem_Hovy.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   mralmubarak yes tried murder way heaven total...      1
1           1   nigzzzk command fear allah quran ritual grove...      1
2           2   ib alkisai islamic documentation based belief...      1
3           3   benfrancisallen : good weeks isis new front o...      0
4           4  self described radical feminist makes death th...      1


In [11]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_hatEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    hatEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    hatEval_2019_model.load_weights("Models/model_hatEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in hatEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    hatEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    hatEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=hatEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f4c0b07db70> False
<keras.layers.recurrent.LSTM object at 0x7f4c0b07d8d0> True
<keras.layers.core.Dense object at 0x7f4c0b07d908> True
<keras.layers.core.Dense object at 0x7f4c8439dac8> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f4c0b07db70> False
<keras.layers.recurrent.LSTM object at 0x7f4c0b07d8d0> False
<keras.layers.core.Dense object at 0x7f4c0b07d908> True
<keras.layers.core.Dense object at 0x7f4c8439dac8> True
Epoch 1/8
2841/2841 [==============================] - 5s 2ms/step - loss: 0.6890 - acc: 0.6829
Epoch 2/8
2841/2841 [==============================] - 6s 2ms/step - loss: 0.6229 - acc: 0.6846
Epoch 3/8
2841/2841 [==============================] - 6s 2ms/step - loss: 0.5934 - acc: 0.7075
Epoch 4/8
2841/2841 [==============================] - 5s 2ms/step - loss: 0.5640 - acc: 0.7209
Epoch 5/8
2841/2841 [==============================] - 5s 2ms/step - loss: 

In [12]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.7175518818149842, 0.7234342012667135, 0.7255453905700211, 0.7294159042927516, 0.7170010559662091]
Mean: 0.722589686782136
Std Dev: 0.0047467653909246815
Macro-F1-Scores: 
[0.6109862733353725, 0.6201413451597105, 0.6203494191076886, 0.6311317960294521, 0.6113992704902415]
Mean: 0.618801620824493
Std Dev: 0.007376966968226477


In [13]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.6869504045022863, 0.6935256861365235, 0.6903589021815623, 0.6896551724137931, 0.6832101372756072]
Mean: 0.6887400605019545
Std Dev: 0.0034682332792219275
Macro-F1-Scores: 
[0.49911195890068194, 0.5134406271971644, 0.503327224617252, 0.4944455738847583, 0.4947892297575842]
Mean: 0.5010229228714882
Std Dev: 0.007004019133108344


In [14]:
# Dataset-3
df=pd.read_csv('../Proper Experiments/Parsed Datasets/OffensEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0                 user ask native americans take is       1
1           1   user user home you re drunk ! ! ! user maga t...      1
2           2  amazon investigating chinese employees selling...      0
3           3   user someone should have taken piece shit vol...      1
4           4   user user obama wanted liberals amp; illegals...      0


In [15]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_hatEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    hatEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    hatEval_2019_model.load_weights("Models/model_hatEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in hatEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    hatEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    hatEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=hatEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f4bf2c77f28> False
<keras.layers.recurrent.LSTM object at 0x7f4c6978d240> True
<keras.layers.core.Dense object at 0x7f4c6978d080> True
<keras.layers.core.Dense object at 0x7f4c84208358> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f4bf2c77f28> False
<keras.layers.recurrent.LSTM object at 0x7f4c6978d240> False
<keras.layers.core.Dense object at 0x7f4c6978d080> True
<keras.layers.core.Dense object at 0x7f4c84208358> True
Epoch 1/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.7578 - acc: 0.6282
Epoch 2/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.6467 - acc: 0.6358
Epoch 3/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.6159 - acc: 0.6710
Epoch 4/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.5959 - acc: 0.6949
Epoch 5/8
2383/2383 [==============================] - 4s 2ms/step - loss: 

In [16]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.6891778523489933, 0.7163239613932019, 0.702895509861519, 0.697440201426773, 0.7007973143096936]
Mean: 0.701326967868036
Std Dev: 0.008835820651586628
Macro-F1-Scores: 
[0.5496413409009039, 0.5958674269107957, 0.5832063557940296, 0.5860886780584791, 0.5700812202546853]
Mean: 0.5769770043837787
Std Dev: 0.015959109099194024


In [17]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.6413590604026845, 0.6496013428451531, 0.6261015526647083, 0.6357532522031053, 0.6328157784305497]
Mean: 0.6371261973092401
Std Dev: 0.00794422890937173
Macro-F1-Scores: 
[0.4879147509402876, 0.5004863831544489, 0.46768148632294965, 0.4776614356479648, 0.4785979150668167]
Mean: 0.48246839422649357
Std Dev: 0.011054870704905463


In [6]:
# Dataset-4
df=pd.read_csv('../Proper Experiments/Parsed Datasets/StormfrontWS.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0  class                                               text
0           0      0  march 13th 2014 booklet downloaded 18 300 time...
1           1      0  order help increase booklets downloads would g...
2           2      0  simply copy paste following text youtube video...
3           3      1  click free download colorfully illustrated 132...
4           4      0              click download 7 42 green banner link


In [7]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

predicted=[]
ft_predicted=[]
actual=[]


i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_hatEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    hatEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    hatEval_2019_model.load_weights("Models/model_hatEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in hatEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    hatEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    hatEval_2019_model.fit(Xft, np.array(yft), epochs = 8)
    
    actual.append(ytest)

    ftm_pred=hatEval_2019_model.predict_classes(Xtest)
    ft_predicted.append(ftm_pred)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    predicted.append(om_pred)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fbc1c3a34a8> False
<keras.layers.recurrent.LSTM object at 0x7fbc1c3a35c0> True
<keras.layers.core.Dense object at 0x7fbc1c3a3780> True
<keras.layers.core.Dense object at 0x7fbc1c373c18> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fbc1c3a34a8> False
<keras.layers.recurrent.LSTM object at 0x7fbc1c3a35c0> False
<keras.layers.core.Dense object at 0x7fbc1c3a3780> True
<keras.layers.core.Dense object at 0x7fbc1c373c18> True
Epoch 1/8
1926/1926 [==============================] - 4s 2ms/step - loss: 0.4035 - acc: 0.8479
Epoch 2/8
1926/1926 [==============================] - 4s 2ms/step - loss: 0.3728 - acc: 0.8775
Epoch 3/8
1926/1926 [==============================] - 4s 2ms/step - loss: 0.3515 - acc: 0.8858
Epoch 4/8
1926/1926 [==============================] - 4s 2ms/step - loss: 0.3294 - acc: 0.8899
Epoch 5/8
1926/1926 [==============================] - 4s 2ms/step - loss: 

In [8]:
from sklearn.metrics import classification_report

print(classification_report(actual[0], ft_predicted[0]))

             precision    recall  f1-score   support

          0       0.89      0.99      0.94      1712
          1       0.47      0.04      0.08       216

avg / total       0.84      0.89      0.84      1928



In [9]:
print(classification_report(actual[0], predicted[0]))

             precision    recall  f1-score   support

          0       0.90      0.90      0.90      1712
          1       0.17      0.17      0.17       216

avg / total       0.81      0.82      0.82      1928



In [20]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.8853734439834025, 0.885773624091381, 0.8852544132917964, 0.8909657320872274, 0.8868120456905504]
Mean: 0.8868358518288716
Std Dev: 0.0021365575115081356
Macro-F1-Scores: 
[0.4870012990875084, 0.48293787737529836, 0.4952177764706301, 0.514488752310758, 0.4833487557095606]
Mean: 0.49259889219075104
Std Dev: 0.011800489919315923


In [21]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.8174273858921162, 0.8276220145379024, 0.8307372793354102, 0.8302180685358256, 0.8431983385254413]
Mean: 0.8298406173653392
Std Dev: 0.008226611705671287
Macro-F1-Scores: 
[0.5336236090953073, 0.5672027290448343, 0.5512209486832739, 0.5508200111973697, 0.5611961917407247]
Mean: 0.5528126979523019
Std Dev: 0.011419996150130179


In [22]:
# Dataset-5
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_1.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      0
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      0
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      0
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      0


In [23]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_hatEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    hatEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    hatEval_2019_model.load_weights("Models/model_hatEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in hatEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    hatEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    hatEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=hatEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f4bde86d048> False
<keras.layers.recurrent.LSTM object at 0x7f4bde86dfd0> True
<keras.layers.core.Dense object at 0x7f4bde86d438> True
<keras.layers.core.Dense object at 0x7f4be5296fd0> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f4bde86d048> False
<keras.layers.recurrent.LSTM object at 0x7f4bde86dfd0> False
<keras.layers.core.Dense object at 0x7f4bde86d438> True
<keras.layers.core.Dense object at 0x7f4be5296fd0> True
Epoch 1/8
4460/4460 [==============================] - 8s 2ms/step - loss: 0.2808 - acc: 0.9253
Epoch 2/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.2306 - acc: 0.9417
Epoch 3/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.2101 - acc: 0.9424
Epoch 4/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.2013 - acc: 0.9426
Epoch 5/8
4460/4460 [==============================] - 7s 2ms/step - loss: 

In [24]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.9428507395786643, 0.9410578216046616, 0.9417040358744395, 0.9439461883408071, 0.9403587443946189]
Mean: 0.9419835059586383
Std Dev: 0.0012789887783756
Macro-F1-Scores: 
[0.5147855211361544, 0.5066141119835189, 0.5070264461011047, 0.5259869674049084, 0.529105619804269]
Mean: 0.5167037332859911
Std Dev: 0.009371393792352554


In [25]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.8930972658000896, 0.898027790228597, 0.8946188340807175, 0.8984304932735426, 0.8975336322869956]
Mean: 0.8963416031339886
Std Dev: 0.0021033915651541314
Macro-F1-Scores: 
[0.5120864610591687, 0.517085208993655, 0.5183625152680139, 0.5099446753981981, 0.5149496237457697]
Mean: 0.5144856968929611
Std Dev: 0.0031106147560124213


In [6]:
# Dataset-6
df=pd.read_csv('../Proper Experiments/Parsed Datasets/GAB.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_hatEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    hatEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    hatEval_2019_model.load_weights("Models/model_hatEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in hatEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    hatEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    hatEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=hatEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  joined gab remind retarded jew haters are typi...
1           1      0       left really scared of https : redd it 9rfkts
2           2      0   that literally looks like monkey supposed pre...
3           3      1                                          dumb cunt
4           4      0                                     makes asshole 
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f68ad9eb5f8> False
<keras.layers.recurrent.LSTM object at 0x7f68ad9ebb38> True
<keras.layers.core.Dense object at 0x7f68ad9ebe48> True
<keras.layers.core.Dense object at 0x7f68ad74edd8> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f68ad9eb5f8> False
<keras.layers.recurrent.LSTM object at 0x7f68ad9ebb38> False
<keras.layers.core.Dense object at 0x7f68ad9ebe48> True
<keras.layers.core.Dense object at 0x7f68ad74edd8> True
Epoch 1/8
6079/6079 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.7164939976977471, 0.7199013157894737, 0.7145912156604705, 0.704885671985524, 0.7131107089981905]
Mean: 0.7137965820262812
Std Dev: 0.005001000056025739
Macro-F1-Scores: 
[0.7040903976387847, 0.7058911667450778, 0.7032581502852736, 0.6973537071078432, 0.6974990213164691]
Mean: 0.7016184886186896
Std Dev: 0.0035274062750278636
Without Fine Tuning: 
Accuracies: 
[0.5637230718631804, 0.5587171052631579, 0.5561770027965126, 0.5512419805889126, 0.5510774798486593]
Mean: 0.5561873280720845
Std Dev: 0.004769786668784826
Macro-F1-Scores: 
[0.459914622892668, 0.4504160613714822, 0.45136759183734637, 0.44044973724927466, 0.44297626963326214]
Mean: 0.44902485659680663
Std Dev: 0.006872102228442216


In [8]:
# Dataset-7
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Reddit.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_hatEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    hatEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    hatEval_2019_model.load_weights("Models/model_hatEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in hatEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    hatEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    hatEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=hatEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  subsection retarded hungarians ohh boy brace l...
1           1      0   hiii got work 444 mainly typa guys imagine wr...
2           2      0                    wow guess soyboys every country
3           3      0   owen benjamin soyboy song goes every country ...
4           4      0   y all hear sumn means live small town for wor...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f68606dcef0> False
<keras.layers.recurrent.LSTM object at 0x7f68606dce80> True
<keras.layers.core.Dense object at 0x7f68606dcda0> True
<keras.layers.core.Dense object at 0x7f68b0085898> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f68606dcef0> False
<keras.layers.recurrent.LSTM object at 0x7f68606dce80> False
<keras.layers.core.Dense object at 0x7f68606dcda0> True
<keras.layers.core.Dense object at 0x7f68b0085898> True
Epoch 1/8
4018/4018 [======

In [9]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8071659616820105, 0.7984071677451469, 0.8076157292185167, 0.8018914883026381, 0.8063713290194127]
Mean: 0.8042903351935451
Std Dev: 0.003578383267837193
Macro-F1-Scores: 
[0.6441924975448257, 0.6377096824034869, 0.6336053371944647, 0.6489681952581543, 0.6631433619499469]
Mean: 0.6455238148701758
Std Dev: 0.010268353107363676
Without Fine Tuning: 
Accuracies: 
[0.716596168201045, 0.719761075161772, 0.7190144350423096, 0.724987555998009, 0.7192633150821304]
Mean: 0.7199245098970531
Std Dev: 0.0027569700761163373
Macro-F1-Scores: 
[0.4847288000995054, 0.4877824558447987, 0.48420425107873366, 0.49391999337976245, 0.4881253498290778]
Mean: 0.4877521700463756
Std Dev: 0.003461523360841762


In [6]:
# Dataset-8
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_2.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_hatEval_2019.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    hatEval_2019_model = model_from_json(loaded_model_json)
    # load weights into new model
    hatEval_2019_model.load_weights("Models/model_hatEval_2019.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in hatEval_2019_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in hatEval_2019_model.layers:
        print(layer, layer.trainable)
    hatEval_2019_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    hatEval_2019_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=hatEval_2019_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      0   ! ! ! mayasolovely : woman complain cleaning ...
1           1      0              momma said pussy cats inside doghouse
2           2      0   addicted2guys : - simplyaddictedtoguys http :...
3           3      0   allaboutmanfeet : http : t co 3gzupfumev woof...
4           4      0   allyhaaaaa : lemmie eat oreo amp; dishes one ...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fdc107e8d68> False
<keras.layers.recurrent.LSTM object at 0x7fdc107e87f0> True
<keras.layers.core.Dense object at 0x7fdc107e8c50> True
<keras.layers.core.Dense object at 0x7fdc1064cd30> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fdc107e8d68> False
<keras.layers.recurrent.LSTM object at 0x7fdc107e87f0> False
<keras.layers.core.Dense object at 0x7fdc107e8c50> True
<keras.layers.core.Dense object at 0x7fdc1064cd30> True
Epoch 1/8
1006/1006 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.7956349206349206, 0.8043694141012909, 0.8170974155069582, 0.8170974155069582, 0.7912524850894632]
Mean: 0.8050903301679181
Std Dev: 0.010674742549636736
Macro-F1-Scores: 
[0.6775471920790603, 0.6966909207108336, 0.6914275045343007, 0.7073824654589143, 0.6829318076759139]
Mean: 0.6911959780918044
Std Dev: 0.010457879751174839
Without Fine Tuning: 
Accuracies: 
[0.7132936507936508, 0.7199602780536246, 0.7286282306163022, 0.7226640159045725, 0.7335984095427436]
Mean: 0.7236289169821788
Std Dev: 0.007008446212861843
Macro-F1-Scores: 
[0.48515348439352945, 0.4813828621520929, 0.4965823292951844, 0.4637844240450104, 0.4969399331263435]
Mean: 0.48476860660243215
Std Dev: 0.012162668680953233
